In [ ]:
!pip -q install -U trackio scikit-learn pandas matplotlib

import os, time, math, json, random, pathlib, itertools, tempfile
from dataclasses import dataclass
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import trackio

In [ ]:
def make_dataset(n=12000, n_informative=18, n_classes=3, seed=42):
    X, y = make_classification(
        n_samples=n, n_features=32, n_informative=n_informative, n_redundant=0,
        n_classes=n_classes, random_state=seed, class_sep=2.0
    )
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=seed)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=seed)
    ss = StandardScaler().fit(X_train)
    return ss.transform(X_train), y_train, ss.transform(X_val), y_val, ss.transform(X_test), y_test

def batches(X, y, bs, shuffle=True, seed=0):
    idx = np.arange(len(X))
    if shuffle:
        rng = np.random.default_rng(seed)
        rng.shuffle(idx)
    for i in range(0, len(X), bs):
        j = idx[i:i+bs]
        yield X[j], y[j]

def cm_table(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    df = pd.DataFrame(cm, columns=[f"pred_{i}" for i in range(cm.shape[0])])
    df.insert(0, "true", [f"true_{i}" for i in range(cm.shape[0])])
    return df

In [ ]:
@dataclass
class RunCfg:
    lr: float = 0.05
    l2: float = 1e-4
    epochs: int = 8
    batch_size: int = 256
    seed: int = 0
    project: str = "trackio-demo"

def train_and_log(cfg: RunCfg, Xtr, ytr, Xva, yva):
    run = trackio.init(
        project=cfg.project,
        name=f"sgd_lr{cfg.lr}_l2{cfg.l2}",
        config={"lr": cfg.lr, "l2": cfg.l2, "epochs": cfg.epochs, "batch_size": cfg.batch_size, "seed": cfg.seed}
    )

    clf = SGDClassifier(loss="log_loss", penalty="l2", alpha=cfg.l2, learning_rate="constant",
                        eta0=cfg.lr, random_state=cfg.seed)
    n_classes = len(np.unique(ytr))
    clf.partial_fit(Xtr[:cfg.batch_size], ytr[:cfg.batch_size], classes=np.arange(n_classes))

    global_step = 0
    for epoch in range(cfg.epochs):
        epoch_losses = []
        for xb, yb in batches(Xtr, ytr, cfg.batch_size, shuffle=True, seed=cfg.seed + epoch):
            clf.partial_fit(xb, yb)
            probs = np.clip(clf.predict_proba(xb), 1e-9, 1 - 1e-9)
            loss = log_loss(yb, probs, labels=np.arange(n_classes))
            epoch_losses.append(loss)
            global_step += 1

        val_probs = np.clip(clf.predict_proba(Xva), 1e-9, 1 - 1e-9)
        val_preds = np.argmax(val_probs, axis=1)
        val_loss = log_loss(yva, val_probs, labels=np.arange(n_classes))
        val_acc  = accuracy_score(yva, val_preds)
        train_loss = float(np.mean(epoch_losses))

        trackio.log({
            "epoch": epoch,
            "train_loss": train_loss,
            "val_loss": val_loss,
            "val_accuracy": val_acc
        })

        if epoch in {cfg.epochs//2, cfg.epochs-1}:
            df = cm_table(yva, val_preds)
            tbl = trackio.Table(dataframe=df)
            trackio.log({f"val_confusion_epoch_{epoch}": tbl})

        time.sleep(0.15)

    trackio.finish()
    return val_acc

In [ ]:
Xtr, ytr, Xva, yva, Xte, yte = make_dataset()
grid = list(itertools.product([0.01, 0.03, 0.1], [1e-5, 1e-4, 1e-3]))
results = []
for lr, l2 in grid:
    acc = train_and_log(RunCfg(lr=lr, l2=l2, seed=123), Xtr, ytr, Xva, yva)
    results.append({"lr": lr, "l2": l2, "val_acc": acc})

summary = pd.DataFrame(results).sort_values("val_acc", ascending=False).reset_index(drop=True)
best = summary.iloc[0].to_dict()
run = trackio.init(project="trackio-demo", name="summary", config={"note": "sweep results"})
trackio.log({"best_val_acc": float(best["val_acc"]), "best_lr": float(best["lr"]), "best_l2": float(best["l2"])})
trackio.log({"sweep_table": trackio.Table(dataframe=summary)})
trackio.finish()

In [1]:
csv_path = "/content/trackio_demo_metrics.csv"
df_csv = pd.DataFrame({
    "step": np.arange(10),
    "metric_x": np.linspace(1.0, 0.2, 10),
    "metric_y": np.linspace(0.1, 0.9, 10),
})
df_csv.to_csv(csv_path, index=False)
trackio.import_csv(csv_path, project="trackio-csv-import")


app = trackio.show(project="trackio-demo")
# trackio.init(project="myproj", space_id="username/trackio-demo-space")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.6/851.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 29.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.2 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompatible.
* Running on public URL: https://76bbc3bd25ff0863eb.gradio.live
* Trackio project initialized: trackio-demo
* Trackio metrics logged to:

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


* Created new run: sgd_lr0.01_l21e-05
* Run finished. Uploading logs to Trackio Space: http://127.0.0.1:7860/ (please wait...)
* Created new run: sgd_lr0.01_l20.0001
* Run finished. Uploading logs to Trackio Space: http://127.0.0.1:7860/ (please wait...)
* Created new run: sgd_lr0.01_l20.001
* Run finished. Uploading logs to Trackio Space: http://127.0.0.1:7860/ (please wait...)
* Created new run: sgd_lr0.03_l21e-05
* Run finished. Uploading logs to Trackio Space: http://127.0.0.1:7860/ (please wait...)
* Created new run: sgd_lr0.03_l20.0001
* Run finished. Uploading logs to Trackio Space: http://127.0.0.1:7860/ (please wait...)
* Created new run: sgd_lr0.03_l20.001
* Run finished. Uploading logs to Trackio Space: http://127.0.0.1:7860/ (please wait...)
* Created new run: sgd_lr0.1_l21e-05
* Run finished. Uploading logs to Trackio Space: http://127.0.0.1:7860/ (please wait...)
* Created new run: sgd_lr0.1_l20.0001
* Run finished. Uploading logs to Trackio Space: http://127.0.0.1:7860/ 